<a href="https://colab.research.google.com/github/ThivyaTS/cnc-colab-data-versioning/blob/main/cnc_anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Kaggle API
!pip install -q kaggle

# Step 2: Upload your Kaggle API key (kaggle.json)
from google.colab import files
print("Please upload your kaggle.json file (get it from Kaggle Account -> Create API Token)")
uploaded = files.upload()  # Click 'Browse' and select your kaggle.json file

# Step 3: Set up Kaggle configuration
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the dataset (Updated for Tool Wear Detection)
!kaggle datasets download -d shasun/tool-wear-detection-in-cnc-mill

# Step 5: Unzip the dataset in Colab
import zipfile
import os

# Create target directory if it doesn't exist
dataset_dir = '/content/cnc_tool_wear_dataset'
os.makedirs(dataset_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile('tool-wear-detection-in-cnc-mill.zip', 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

# Step 6: Verify the files
print("\nDownloaded files in Colab storage:")
!ls -l {dataset_dir}

# Step 7: (Optional) Save to Google Drive for persistence
save_to_drive = False  # Change to True if you want to save to Drive
if save_to_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    !cp -r {dataset_dir} '/content/drive/MyDrive/'
    print("\nDataset also saved to Google Drive")
else:
    print("\nDataset stored temporarily in Colab at:", dataset_dir)

Please upload your kaggle.json file (get it from Kaggle Account -> Create API Token)


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/shasun/tool-wear-detection-in-cnc-mill
License(s): CC0-1.0
  0% 0.00/2.57M [00:00<?, ?B/s]
100% 2.57M/2.57M [00:00<00:00, 664MB/s]

Downloaded files in Colab storage:
total 11984
-rw-r--r-- 1 root root  467193 Jun 10 13:18 experiment_01.csv
-rw-r--r-- 1 root root  736459 Jun 10 13:18 experiment_02.csv
-rw-r--r-- 1 root root  671985 Jun 10 13:18 experiment_03.csv
-rw-r--r-- 1 root root  234196 Jun 10 13:18 experiment_04.csv
-rw-r--r-- 1 root root  202320 Jun 10 13:18 experiment_05.csv
-rw-r--r-- 1 root root  574296 Jun 10 13:18 experiment_06.csv
-rw-r--r-- 1 root root  250860 Jun 10 13:18 experiment_07.csv
-rw-r--r-- 1 root root  267820 Jun 10 13:18 experiment_08.csv
-rw-r--r-- 1 root root  328887 Jun 10 13:18 experiment_09.csv
-rw-r--r-- 1 root root  574595 Jun 10 13:18 experiment_10.csv
-rw-r--r-- 1 root root 1022545 Jun 10 13:18 experiment_11.csv
-rw-r--r-- 1 root root 1004981 Jun 10 13:18 experiment_12.cs